In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
pd.options.display.float_format = '{:.5f}'.format

# 데이터 불러오기

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
car_2020 = pd.read_csv('data/car_2020.csv',encoding='cp949')
age = pd.read_csv('data/age_gender_info.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [3]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72000,134,38.00000,A,15667000,103680,0.00000,3.00000,1425.00000,1015.00000
1,C2483,900,아파트,경상북도,국민임대,39.72000,15,38.00000,A,15667000,103680,0.00000,3.00000,1425.00000,1015.00000
2,C2483,900,아파트,경상북도,국민임대,51.93000,385,38.00000,A,27304000,184330,0.00000,3.00000,1425.00000,1015.00000
3,C2483,900,아파트,경상북도,국민임대,51.93000,15,38.00000,A,27304000,184330,0.00000,3.00000,1425.00000,1015.00000
4,C2483,900,아파트,경상북도,국민임대,51.93000,41,38.00000,A,27304000,184330,0.00000,3.00000,1425.00000,1015.00000


In [4]:
train['총세대수'] = np.log(train['총세대수'])

In [5]:
test['총세대수'].value_counts()

2572    46
1161    31
1021    27
1505    27
627     25
        ..
656      2
638      2
558      2
420      2
494      2
Name: 총세대수, Length: 144, dtype: int64

In [6]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,6.80239,아파트,경상북도,국민임대,39.72000,134,38.00000,A,15667000,103680,0.00000,3.00000,1425.00000,1015.00000
1,C2483,6.80239,아파트,경상북도,국민임대,39.72000,15,38.00000,A,15667000,103680,0.00000,3.00000,1425.00000,1015.00000
2,C2483,6.80239,아파트,경상북도,국민임대,51.93000,385,38.00000,A,27304000,184330,0.00000,3.00000,1425.00000,1015.00000
3,C2483,6.80239,아파트,경상북도,국민임대,51.93000,15,38.00000,A,27304000,184330,0.00000,3.00000,1425.00000,1015.00000
4,C2483,6.80239,아파트,경상북도,국민임대,51.93000,41,38.00000,A,27304000,184330,0.00000,3.00000,1425.00000,1015.00000


car_2020 = car_2020.drop(['시군구(1)'], axis=1)
car_2020.columns = ['지역','월','항목','분류','자동차수']
car_2020['월'] = pd.to_datetime(car_2020['월']).dt.month
car_2020.head()

In [7]:
train.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

# 컬렴명 바꿔주기

In [8]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]


# 지하철 Y or N로 변경

In [9]:
train['지하철'] = train['지하철'].fillna(0)
train['지하철'] = np.where(train['지하철'] == 0, 0, 1)

# 결측치 처리

In [10]:
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
train.loc[train.임대료=='-', '임대료'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
train['임대료'] = train['임대료'].astype(float)
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)

In [11]:
train['버스'] = train['버스'].fillna(train['버스'].mean())

In [12]:
train.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
신분          0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
등록차량수       0
dtype: int64

# 임대건물구분 : 아파트->상가 / 전용면적별세대수 : x->1

In [13]:
idx = train[(train['임대건물구분']=='아파트') & (train['신분']=='D')]['전용면적별세대수'].index
train.loc[idx, '전용면적별세대수'] = 1
train.loc[idx, '임대건물구분'] = '상가'

In [14]:
train[(train['임대건물구분']=='상가') & (train['신분']=='D')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
88,C1925,6.39859,상가,강원도,임대상가,32.10000,1,9.00000,D,0.00000,0.00000,0,4.00000,117.00000,75.00000
89,C1925,6.39859,상가,강원도,임대상가,32.10000,1,9.00000,D,0.00000,0.00000,0,4.00000,117.00000,75.00000
90,C1925,6.39859,상가,강원도,임대상가,32.10000,1,9.00000,D,0.00000,0.00000,0,4.00000,117.00000,75.00000
91,C1925,6.39859,상가,강원도,임대상가,72.16000,1,9.00000,D,0.00000,0.00000,0,4.00000,117.00000,75.00000
101,C1874,6.42811,상가,충청남도,임대상가,12.62000,1,2.00000,D,0.00000,0.00000,0,2.00000,97.00000,62.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,C1350,7.24494,상가,대전광역시,공공분양,74.94000,1,2.00000,D,0.00000,0.00000,0,6.00000,1636.00000,2315.00000
2334,C1350,7.24494,상가,대전광역시,공공분양,84.94000,1,2.00000,D,0.00000,0.00000,0,6.00000,1636.00000,2315.00000
2335,C1350,7.24494,상가,대전광역시,공공분양,84.94000,1,2.00000,D,0.00000,0.00000,0,6.00000,1636.00000,2315.00000
2336,C1350,7.24494,상가,대전광역시,공공분양,84.96000,1,2.00000,D,0.00000,0.00000,0,6.00000,1636.00000,2315.00000


In [15]:
# 원핫 인코딩
#train = pd.get_dummies(data = train, columns = ['지하철'])

# 지역명 숫자로 매핑

In [16]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i
    
train['지역'] = train['지역'].map(local_map)

# 전용면적을 5의 배수로 변경

In [17]:
train['전용면적'] = train['전용면적']//5*5

# 전용면적 상/하한 적용

In [18]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      2952 non-null   object 
 1   총세대수      2952 non-null   float64
 2   임대건물구분    2952 non-null   object 
 3   지역        2952 non-null   int64  
 4   공급유형      2952 non-null   object 
 5   전용면적      2952 non-null   float64
 6   전용면적별세대수  2952 non-null   int64  
 7   공가수       2952 non-null   float64
 8   신분        2952 non-null   object 
 9   임대보증금     2952 non-null   float64
 10  임대료       2952 non-null   float64
 11  지하철       2952 non-null   int32  
 12  버스        2952 non-null   float64
 13  단지내주차면수   2952 non-null   float64
 14  등록차량수     2952 non-null   float64
dtypes: float64(8), int32(1), int64(2), object(4)
memory usage: 334.5+ KB


# test 불러오기

In [20]:
test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [21]:
test.isnull().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
신분            2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
dtype: int64

In [22]:
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)
test['임대보증금'] = test['임대보증금'].astype(float)
test['임대료'] = test['임대료'].astype(float)

In [23]:
test['지하철'].value_counts()

0.00000    881
1.00000     64
2.00000     35
Name: 지하철, dtype: int64

In [24]:
test['지하철'] = test['지하철'].fillna(0)
test['지하철'] = np.where(test['지하철'] == 0, 0, 1)

In [25]:
test.loc[test.단지코드.isin(['C2411']) & test.신분.isnull(), '신분'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.신분.isnull(), '신분'] = 'C'

In [26]:
local_map = {}
for i, loc in enumerate(test['지역'].unique()):
    local_map[loc] = i
    
test['지역'] = test['지역'].map(local_map)

In [27]:
test['전용면적'] = test['전용면적']//5*5

idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [28]:
test.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
신분          0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
dtype: int64

In [29]:
columns = ['총세대수', '지역', '전용면적별세대수', '공가수', '지하철', '버스', '단지내주차면수']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [30]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [31]:
new_train['임대보증금'] = 0
new_train['임대료'] = 0
new_test['임대보증금'] = 0
new_test['임대료'] = 0

## 수정한 코드

In [32]:
# Train Dataframe Set
# 미사용 열s - 임대건물구분, 공급유형, 자격유형 임대보증금, 임대료
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0]) # index 재설정
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col] # Fixed Data
    
    deposit_cost = []
    rental_cost = []

    for col in area_columns:
        area = float(col.split('_')[-1]) # 면적의 숫자를 float형으로 저장
        # '면적_10' 등의 열을 만들고 - 같은 전용면적들 끼리 전용면적별세대수의 합을 Value로 Cell에 입력
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
        
        try:
          deposit_cost.append(temp[temp['전용면적']==area]['임대보증금'][0])
          rental_cost.append(temp[temp['전용면적']==area]['임대료'][0])
        except KeyError as e:
          continue
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    # print(type(deposit_cost[0]))
    new_train.loc[i, '임대보증금'] = int(np.mean(deposit_cost))
    new_train.loc[i, '임대료'] = int(np.mean(rental_cost))

# Test DataFrame Set
# 미사용 열s - 임대건물구분, 공급유형, 자격유형 임대보증금, 임대료, 등록차량수
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]

    deposit_cost = []
    rental_cost = []
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
        
        try:
          deposit_cost.append(temp[temp['전용면적']==area]['임대보증금'][0])
          rental_cost.append(temp[temp['전용면적']==area]['임대료'][0])
        except KeyError as e:
          continue
          
    new_test.loc[i, '임대보증금'] = int(np.mean(deposit_cost))
    new_test.loc[i, '임대료'] = int(np.mean(rental_cost))


# 추가해볼 것 임대보증금 , 임대료 Column 만들고 평균

423it [00:07, 59.14it/s]
150it [00:02, 60.81it/s]


In [57]:
x_train = new_train.iloc[:, :-1]
y_train = new_train.iloc[:,-1]
x_test = new_test

In [50]:
forest = RandomForestRegressor(n_jobs=-1, random_state=42)
forest.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [73]:
pred = lgb.predict(x_test)
#pred

In [74]:
submission['num'] = pred

In [75]:
submission.to_csv('0717.csv', index=False)

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

x = new_train.iloc[:, :-1]
y = new_train.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=0)

forest = RandomForestRegressor(random_state=0)
forest.fit(x_train, y_train)

pred = forest.predict(x_test)
mean_absolute_error(y_test, pred)

130.2770754716981